# Analysis of the **getLebedevSphear()** function for MATLAB
---
A description from the [documenation](https://www.mathworks.com/matlabcentral/fileexchange/27097-getlebedevsphere):

for Lebedev quadratures on the surface of the unit sphere at double precision.

**********Relative error is generally expected to be ~2.0E-14 [1]********

Lebedev quadratures are superbly accurate and efficient quadrature rules for approximating integrals of the form $v = \iint_{4\pi} f(\Omega) \  d\Omega$, where $\Omega$ is the solid angle on the surface of the unit sphere. Lebedev quadratures integrate all spherical harmonics up to $l = order$, where $degree \approx order(order+1)/3$. These grids may be easily combined with radial quadratures to provide robust cubature formulae. For example, see 'A. Becke, 1988c, J. Chem. Phys., 88(4), pp. 2547' (The first paper on tractable molecular Density Functional Theory methods, of which Lebedev grids and numerical cubature are an intrinsic part).

@param degree - positive integer specifying number of points in the requested quadrature. Allowed values are (degree -> order):
degree: { 6, 14, 26, 38, 50, 74, 86, 110, 146, 170, 194, 230, 266, 302, 350, 434, 590, 770, 974, 1202, 1454, 1730, 2030, 2354, 2702, 3074, 3470, 3890, 4334, 4802, 5294, 5810 };
order: {3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,35,41,47,53,59,65,71,77, 83,89,95,101,107,113,119,125,131};

There is an example where 
$$\int_S x^2+y^2-z^2 \, d\Omega = 4.188790204786399$$

---
#### This is the part where we come in
---
First of all, it's kind of a weird way to format it. It should rather be

$$\int_S x^2+y^2-z^2 \, dx \, dy \, dz = 4.188790204786399$$

But how is it calculated?

## Parametric representation to solve the surface integral

$$
\begin{aligned}
x &= r \, sin(\phi) cos(\theta)  \\
y &= r \, sin(\phi) sin(\theta)  \\
z &= r \, cos(\phi)
\end{aligned}
$$

where

$$0 < \phi < \pi, \; 0 < \theta < 2\pi.$$

## Volume element

### Method 1: Determinant of a Jacobian
$$J = 
\begin{vmatrix} 
    \frac{\partial x}{\partial r} & \frac{\partial x}{\partial \phi} & \frac{\partial x}{\partial \theta} \\
    \frac{\partial y}{\partial r} & \frac{\partial y}{\partial \phi} & \frac{\partial y}{\partial \theta} \\
    \frac{\partial z}{\partial r} & \frac{\partial z}{\partial \phi} & \frac{\partial z}{\partial \theta}
\end{vmatrix} 
=
\begin{vmatrix} 
    \sin(\theta)\cos(\phi)   & -r \, \sin(\phi)   \sin(\theta) & r \, \cos(\phi) \cos(\theta) \\
    \sin(\phi)  \sin(\theta) &  r \, \sin(\theta) \cos(\phi)   & r \, \sin(\phi) \cos(\theta) \\
    \cos(\theta)             &  0                              &-r \, \sin(\theta)
\end{vmatrix} = 
r^2 \, \sin(\phi)
$$

$$\int_{S}  f(\phi, \theta) \mathrm{J} \, d\phi \, d\theta, \quad 0 < \phi < \pi; \; 0 < \theta < 2\pi$$

---

### Method 2: Cross product of the Tangent Vectors

$$
T_{\phi} \times T_{\theta} = 
\begin{bmatrix} 
    \hat{i}                      &  \hat{j}                          & \hat{k} \\
    \frac{\partial x}{\partial \phi} & \frac{\partial y}{\partial \phi} & \frac{\partial z}{\partial \phi} \\
    \frac{\partial x}{\partial \theta} & \frac{\partial y}{\partial \theta} & \frac{\partial z}{\partial \theta}
\end{bmatrix}
$$

$$|T_{\phi} \times T_{\theta}| = r^2 \, sin(\phi)$$


Therefore the final inetgral in spherical coornates can be caluclated by

$$\int_{S}  f(\phi, \theta) |T_{\phi} \times T_{\theta}| \, d\phi \, d\theta, \quad 0 < \phi < \pi; \; 0 < \theta < 2\pi$$

---

## Putting it all together

We are dealing with a unit sphear therefore r = 1.

$$\int_{0}^{2 \pi} \int_{0}^{\pi} \bigg[ \left( (r \, \sin(\phi) \cos(\theta))^2 + (r \, \sin(\phi) \sin(\theta))^2 - (r \, \cos(\phi))^2 \right)  \cdot r^2 \, \sin (\phi) \bigg] \; d\phi d\theta = 4.188790204786399$$

---
Three different methods of this are shown below. They are
1. SymPy method
2. SciPy method
3. Lebedev quadratures method
---

In [3]:
#******************************************************************
# This code is released under the GNU General Public License (GPL).
#******************************************************************

In [4]:
import h5py
import time as t
import numpy as np
import numba as nb
import scipy as sp
import sympy as sy
import lebedev as l

## SymPy method

In [5]:
r, phi, theta = sy.symbols("r, phi, theta")
phi

phi

In [6]:
# Cartesian to Spherical coord.
x = r * sy.sin(phi) * sy.cos(theta)
y = r * sy.sin(phi) * sy.sin(theta)
z = r * sy.cos(phi)

# Find the jacobian matrix
f = sy.Matrix([x, y, z])
v = sy.Matrix([r, phi, theta])
J = f.jacobian(v)
J

Matrix([
[sin(phi)*cos(theta), r*cos(phi)*cos(theta), -r*sin(phi)*sin(theta)],
[sin(phi)*sin(theta), r*sin(theta)*cos(phi),  r*sin(phi)*cos(theta)],
[           cos(phi),           -r*sin(phi),                      0]])

In [7]:
det_J = sy.simplify(sy.det(J))
det_J

r**2*sin(phi)

In [8]:
T_phi   = sy.Matrix([x.diff(phi), y.diff(phi), z.diff(phi)])
T_theta = sy.Matrix([x.diff(theta), y.diff(theta), z.diff(theta)])
Tp_x_Tt = T_phi.cross(T_theta)
Tp_x_Tt

Matrix([
[                                                r**2*sin(phi)**2*cos(theta)],
[                                                r**2*sin(phi)**2*sin(theta)],
[r**2*sin(phi)*sin(theta)**2*cos(phi) + r**2*sin(phi)*cos(phi)*cos(theta)**2]])

In [9]:
sy.simplify(sy.sqrt(Tp_x_Tt[0]**2 + Tp_x_Tt[1]**2 + Tp_x_Tt[2]**2))

sqrt(r**4*sin(phi)**2)

In [10]:
r = 1
func = ((r*sy.sin(phi)*sy.cos(theta))**2 + (r*sy.sin(phi)*sy.sin(theta))**2 - (r*sy.cos(phi))**2 )* r**2*sy.sin(phi)
sy.integrate(func, (phi, 0, sy.pi), (theta, 0, 2*sy.pi))

4*pi/3

In [11]:
# Analytical solution
4*np.pi/3

4.1887902047863905

## SciPy method

In [12]:
# Define the integrand
def integrand(phi, theta):
    return ((np.sin(phi)*np.cos(theta))**2 + (np.sin(phi)*np.sin(theta))**2 - (np.cos(phi))**2 )* np.sin(phi)

# Define the limits of integration
phi_limits   = [0, 2*np.pi]
theta_limits = [0, np.pi]

# Perform the double integration
result, error = sp.integrate.nquad(integrand, [theta_limits, phi_limits])

print("Numerical result:", result)
print("Error:", error)

Numerical result: 4.188790204786391
Error: 2.4797939807798317e-13


## Lebedev quadratures method

In [13]:
# Using a custom module, we get the fastest approach
def f(x, y, z):
    return x**2 + y**2 - z**2
leb = l.getLebedevSphere(590)
v = f(leb['x'], leb['y'], leb['z'])
int = sum(v * leb['w'])
int

4.188790204786399